In [ ]:
pip install -r requirements.txt

In [2]:
# Import
import gensim.downloader

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from statistics import mean

import numpy as np
import pandas as pd

import _helpers as hp

# Load Data
For chosing which ASR data change the variable by the key in the paths dictionary

In [3]:
# Defining paths for our data. "ASR" means the TDNNF-LFMMI method, and "new_ASR" means the Wave2vec method.
paths = {
    "ground_truth": "snips/merged_GT_data.csv",
    "ASR": "snips/old_ASR_data.csv",
    "new_ASR": "snips/new_ASR_with_labels.csv",
    "new_ASR_Autocorrect": "snips/new_ASR_Autocorrection_with_labels.csv",
}

# Chosing which ASR data we will use
ASR_data = pd.read_csv(paths["new_ASR"])

# Groundtruth data
GT_data = pd.read_csv(paths["ground_truth"])

## Data manipulation

In [5]:
# Get ASR data into a numpy word array per sentence plus a numpy label array

XX_ASR = ASR_data["transcript"].apply(lambda x: x.split(" "))
X_ASR = list(XX_ASR)  # numpy word array per transcript

y_num_ASR = np.array(ASR_data["user_action_num"])  # labels

In [6]:
# Get Groundtruth data into a numpy word array per sentence plus a numpy label array

XX_GT = GT_data["transcript"].apply(lambda x: x.split(" "))
X_GT = list(XX_GT)  # numpy word array per transcript

y_num_GT = np.array(GT_data["user_action_num"])  # labels

# Word2Vec

## Import pre-trained Word2Vec model

In [7]:
model_w2v = gensim.downloader.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


## Get features and labels

In [8]:
def get_Word2Vec_feature(sentence):
    """return word2vec numpy array representation of sentence"""

    value_iter = np.zeros((model_w2v.vector_size,))

    for word in sentence:
        try:
            word_vec = model_w2v[word]
            value_iter += np.array(word_vec)
        except:
            continue

    return value_iter

In [9]:
# features Space
features_ASR = np.stack(XX_ASR.apply(get_Word2Vec_feature).values, axis=0)

# features Space
features_GT = np.stack(XX_GT.apply(get_Word2Vec_feature).values, axis=0)

In [10]:
labels_ASR = ASR_data["user_action_num"]
labels_GT = GT_data["user_action_num"]

# Classifiers

## Train/Test Split

The function beneath provides the features and labels needed for testing. Using the loaded ASR or not (then using ground truth data) is decided by input. As standard we use the ASR dataset.

In [11]:
def get_train_test_data(type_of_dataset="ASR", train_size=0.9):
    """Retrieves the relevant dataset and splits according to parameter"""
    # If ASR, give ASR features and labels
    if type_of_dataset == "ASR":
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_ASR, labels_ASR, train_size=train_size
        )
    # If the dataset is not the ASR data, use the ground truth data
    else:
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_GT, labels_GT, train_size=train_size
        )

    return train_features, test_features, train_labels, test_labels


def run_classifier(
    classifier_pipe, type_of_dataset="ASR", train_size=0.9, number_of_times=100
):
    """For running the classifiers multiple times, and returning mean accuracy score. Wraps around get_train_test_data"""
    mean_score_list = []
    n = number_of_times
    for i in range(n):
        train_features, test_features, train_labels, test_labels = get_train_test_data(
            type_of_dataset="ASR"
        )
        classifier_pipe.fit(train_features, train_labels)

        classifier_pred_labels = classifier_pipe.predict(test_features)  # predictions

        classifier_score = classifier_pipe.score(test_features, test_labels)  # accuracy

        mean_score_list.append(classifier_score)
    mean_score = mean(mean_score_list)
    return mean_score, classifier_pred_labels, classifier_score, test_labels

## Logistic Regression

In [12]:
lgr = LogisticRegression(C=0.06, max_iter=1000)  # Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr)  # Scale feature space

mean_score, lgr_pred_labels, lgr_score, test_labels = run_classifier(
    classifier_pipe=lgr_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.795


### Model Evaluation
Gives detail on the last run of the classifier

In [13]:
print(
    classification_report(
        test_labels,
        lgr_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.78      0.74      0.76        34
     SwitchLightOn       0.72      0.82      0.77        28
IncreaseBrightness       0.68      0.70      0.69        27
DecreaseBrightness       0.71      0.69      0.70        29
SetLightBrightness       0.86      0.73      0.79        26
     SetLightColor       0.67      0.73      0.70        22

          accuracy                           0.73       166
         macro avg       0.74      0.73      0.73       166
      weighted avg       0.74      0.73      0.74       166

[[25  3  3  2  0  1]
 [ 2 23  0  1  0  2]
 [ 1  1 19  3  1  2]
 [ 2  1  3 20  1  2]
 [ 1  1  2  2 19  1]
 [ 1  3  1  0  1 16]]

ACCURACY: 0.7349397590361446


# Naive Bayes

### Gaussian Naive Bayes

In [14]:
gnb = GaussianNB()  # Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb)  # Scale feature space

mean_score, gnb_pred_labels, gnb_score, test_labels = run_classifier(
    classifier_pipe=gnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.585


#### Model Evaluation

In [15]:
print(
    classification_report(
        test_labels,
        gnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.52      0.71      0.60        24
     SwitchLightOn       0.67      0.78      0.72        23
IncreaseBrightness       0.58      0.41      0.48        27
DecreaseBrightness       0.47      0.33      0.39        24
SetLightBrightness       0.71      0.81      0.75        36
     SetLightColor       0.86      0.78      0.82        32

          accuracy                           0.65       166
         macro avg       0.63      0.64      0.63       166
      weighted avg       0.65      0.65      0.64       166

[[17  1  0  2  4  0]
 [ 3 18  0  0  2  0]
 [ 4  3 11  4  4  1]
 [ 6  1  7  8  0  2]
 [ 2  1  0  3 29  1]
 [ 1  3  1  0  2 25]]

ACCURACY: 0.6506024096385542


# SVM

In [16]:
svm = SVC()  # Create the classification model

svm_pipe = make_pipeline(preprocessing.Normalizer(), svm)  # Scale feature space

mean_score, svm_pred_labels, svm_score, test_labels = run_classifier(
    classifier_pipe=svm_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.763


### Model Evaluation

In [17]:
print(
    classification_report(
        test_labels,
        svm_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.70      0.58      0.64        24
     SwitchLightOn       0.79      0.84      0.82        32
IncreaseBrightness       0.59      0.71      0.64        24
DecreaseBrightness       0.58      0.56      0.57        27
SetLightBrightness       0.91      0.94      0.92        32
     SetLightColor       0.96      0.85      0.90        27

          accuracy                           0.76       166
         macro avg       0.75      0.75      0.75       166
      weighted avg       0.76      0.76      0.76       166

[[14  3  4  3  0  0]
 [ 2 27  1  1  1  0]
 [ 2  0 17  4  1  0]
 [ 1  2  7 15  1  1]
 [ 1  0  0  1 30  0]
 [ 0  2  0  2  0 23]]

ACCURACY: 0.7590361445783133


# NEURAL NETWORK

In [18]:
mlp = MLPClassifier(
    hidden_layer_sizes=(400, 100), activation="relu", solver="adam", max_iter=5000
)  # Create the classification model

mlp_pipe = make_pipeline(preprocessing.Normalizer(), mlp)  # Scale feature space

mean_score, mlp_pred_labels, mlp_score, test_labels = run_classifier(
    classifier_pipe=mlp_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.794


### Model Evaluation

In [19]:
print(
    classification_report(
        test_labels,
        mlp_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.79      0.82      0.81        33
     SwitchLightOn       0.86      0.78      0.82        32
IncreaseBrightness       0.86      0.68      0.76        28
DecreaseBrightness       0.67      0.67      0.67        27
SetLightBrightness       0.83      0.93      0.88        27
     SetLightColor       0.67      0.84      0.74        19

          accuracy                           0.78       166
         macro avg       0.78      0.79      0.78       166
      weighted avg       0.79      0.78      0.78       166

[[27  2  1  2  1  0]
 [ 1 25  1  2  0  3]
 [ 3  0 19  4  1  1]
 [ 3  1  0 18  2  3]
 [ 0  0  0  1 25  1]
 [ 0  1  1  0  1 16]]

ACCURACY: 0.7831325301204819


# Try Your Self

In [20]:
def user_friendly(sentence, cls):
    """return action from sentence"""

    sent = sentence.split()
    new_sent = []

    for word in sent:
        new_sent.append(hp.autocorrection(word))

    x_pred = get_Word2Vec_feature(new_sent).reshape(1, -1)
    y_pred = cls.predict(x_pred)
    return hp.indx2action(y_pred)

In [21]:
user_friendly("I want to set brithenss to fifty in my living room", lgr_pipe)

['SetLightBrightness']